# Web scrape for triathlon results from various websites (Race Roster and Sportstats)

Lindsay Brin
start 2018 July 6

## Updates
This version builds on the previous versions by adding a function to read Sportstats data for the Sydenham triathlon.

## Previous versions
Based on `raceroster_webscrape.ipynb`, started 2018 June 25, which read Race Roster data for Hampton and Rockwood races in 2017 and 2018 (different header formats).
...which was based on initial scrape of Hampton Ladies Triathlon 2018 results in `raceroster_webscrape_hampton.pynb`.

## Notes

Adding additional data for raceroster format:

* If headers follow the same general approach as previous data:
    * Add info to `race_info` dictionary.
        * Confirm that URL allows for all data to be shown on that page (e.g., if limit is 500 rows shown, make sure there are not more than 500 participants).
    * Add header cleanup to `header_replacement` dictionary as necessary.
* If headers follow some new, creative format:
    * All bets are off. Figure out if `get_rr_table()` works and then go from there.  Once it works, we should be back in the territory of the previous bullet point.

## Setup

Import libraries:

In [2]:
# Import libraries
from lxml import html
from lxml import etree
import requests
import pandas as pd
import numpy as np

## Read in data as table

### Race Roster function

Function to get (single) table from website, extract (multi-line) headers and body, and combine as a pandas dataframe:

In [85]:
# Headers are only combined if the colspan value in the first row > 1.
# This means that for 2018 races where the header value is repeated in both rows, only the first row value is used.  
# Specifically, the second row value is set to "" because colspan == 1.
def get_rr_table(url):
    page = requests.get(url)
    tree = html.fromstring(page.content)
    
    for table in tree.xpath('//table[@class="results-listing__table table table-hover table-striped"]'):
        # Get both header rows
        header1 = [th.text_content().strip().lower() for th in table.xpath('//thead/tr[1]/th')]
        header2 = [th.text_content().strip().lower() for th in table.xpath('//thead/tr[2]/th')]
        # Get colspan (and rowspan?) attributes/values, then convert instances of None to 1; also, convert to integers
        colspan1 = [th.get("colspan") for th in table.xpath('//thead/tr[1]/th')]
        colspan1_int = [1 if element == None else int(element) for element in colspan1]
        # Make expanded header 1 (repeat elements as necessary)
        header1_expanded = [item for item, count in zip(header1, colspan1_int) for i in range(count)]
        # Make corrected header 2 (align elements to header 1)
        # For each element in colspan1_int, add an empty string if the orignal value was None,
        #   and select the appropriate number of elements from header2 if colspan1_int >1.
        #   I could have done this as simply appending elements from header2, but I wanted to 
        #   write it so that it might still make sense with a different race roster header pattern.
        header2_expanded = []  # Initialize empty list
        strloc = 0  # Initialize counter variable
        for i in colspan1_int:
            if i == 1:
                header2_expanded.append('')
            elif i > 1:
                header2_expanded.extend(['-' + element for element in header2[strloc:strloc+i]]) # extend concatenates elements from the list, rather than appending a list (as a sub-list)
                strloc += i    
        # Combine header rows
        header = [h1 + h2 for h1, h2 in zip(header1_expanded, header2_expanded)]
        # Get body of table; remove white spaces and \n; only keep the rows that are the right number of elements
        data = [[td.text_content() for td in tr.xpath('td')]  
                for tr in table.xpath('//tr')]
        for lst in np.arange(start = 0, stop = (len(data)), step = 1):
            data[lst] = [element.strip() for element in data[lst]]
        data = [row for row in data if len(row)==len(header)]
        # Convert to pandas dataframe
        data = pd.DataFrame(data, columns=header)
        return(data)  # Return pandas dataframe

Set variables for each race:

In [4]:
race_info = {'Hampton2017': {
                 'url': 'https://results.raceroster.com/results/sj47pnd6egmunhjt', 
                 'dfname': 'hampton17sprint', 
             }, 
             'Hampton2018': {
                 'url': 'https://results.raceroster.com/results/wjvz7sruf3ngamgq', 
                 'dfname': 'hampton18sprint', 
             },
             'Rockwood2017': {
                 'url': 'https://results.raceroster.com/results/7uqq4njwwzqnbn6q', 
                 'dfname': 'rockwood17sprint',
             },
             'Rockwood2018': {
                 'url': 'https://results.raceroster.com/results/syf4m4gy6sknmzc3?sub_event=13620&query_string=&gender_code=&per_page=500&division=&page=1', 
                 'dfname': 'rockwood18sprint', 
             } 
            }

Read tables for the above urls:

In [5]:
for race in list(race_info.keys()):
    race_info[race]['df'] = get_rr_table(race_info[race]['url'])

Check shapes; number of rows is correct. 

In [605]:
for race in list(race_info.keys()):
    print(race, ": ", race_info[race]['df'].shape, sep = '')

Hampton2017: (195, 17)
Hampton2018: (217, 12)
Rockwood2017: (96, 22)
Rockwood2018: (76, 12)


### Sportstats function

Function to get (single) table from website, extract (multi-line) headers and body, and combine as a pandas dataframe:

In [84]:
def get_sportstats_table(url):
    page = requests.get(url)
    tree = html.fromstring(page.content)

    for table in (tree.xpath('//table[@class="results overview-result"]')):
        # Get header row
        header = [element.text_content().strip().lower() for element in table.xpath('//thead[@id="mainForm:dataTable_head"]/tr/th[@role="columnheader"]')]
        # Get body of table; remove white spaces and \n; only keep the rows that are the right number of elements
        # Note that this gets all rows of all tables. I can't seem to only grab the right one.  
        #  However, the second table doesn't have the same number of columns, so it isn't included in the df.
        data = [[td.text_content() for td in tr.xpath('td')]  
                for tr in table.xpath('//tr')]
        for lst in np.arange(start = 0, stop = (len(data)), step = 1):
            data[lst] = [element.strip() for element in data[lst]]
        data = [row for row in data if len(row)==len(header)]
        # Convert to pandas dataframe
        data = pd.DataFrame(data, columns=header)
        return(data)  # Return pandas dataframe

In [86]:
get_sportstats_table('https://www.sportstats.ca/display-results.xhtml?raceid=43857&status=results')

,comp.,view,bib,name,category,rank,gender place,cat. place,swim,bike,run,finish
0,,,108,Jeff SEABY,M45-49,1,1,1,00:11:06,00:33:40,00:20:20,01:05:05.0
1,,,98,Aaron POIRIER,M19-,2,2,1,00:10:00,00:35:03,00:21:57,01:06:57.9
2,,,176,Justin JAGER,M30-34,3,3,1,00:11:16,00:33:17,00:25:10,01:09:41.9
3,,,119,Nick VANDERSCHOOR,M19-,4,4,2,00:09:04,00:39:34,00:22:28,01:11:04.2
4,,,93,Paul MONROE,M60-64,5,5,1,00:11:56,00:35:00,00:24:50,01:11:44.5
5,,,179,Colin RUNIONS,M19-,6,6,3,00:14:12,00:35:23,00:22:55,01:12:29.5
6,,,69,Anthony ESPOSTI,M50-54,7,7,1,00:12:15,00:35:05,00:25:35,01:12:53.9
7,,,159,Katie WEBB,F35-39,8,1,1,00:22:21,00:41:04,00:10:58,01:14:21.8
8,,,162,Patrick GILMOUR,M45-49,9,8,2,00:11:55,00:38:39,00:24:04,01:14:35.5
9,,,121,Lucas WIENS,M19-,10,9,4,00:12:30,00:37:43,00:25:23,01:15:34.1


## Data cleaning, etc.

Tasks:
* Standardize headers between tables
    * The two-row headers for 2017 races add quite a bit of cleanup/replacement here.
* Calculate decimal times to do calculations downstream
    * This includes renaming original sport columns to include `_hhmmss`
* Pull `gender` out of `division`

### Header standardization

At least for Rockwood and Hampton 2018, the headers are the same for:
* place
* finish
* name
* city
* no.
* swim
* bike
* run
* t1
* t2

The ones that vary are:
* `division` vs. `div`
* `division place` vs. `div place`

For 2017, there is a lot more cleanup that needs to be done. There are also variables that are not in the 2018 data that I am just not working with for now (e.g., `-rank` and `-place` for the different sports).

Logic:
* If header contains both `div` and `place`, make it `division place`
* If header is `div`, change to `division`
* 2017-relevant changes:
    * `bib` -> `no.`
    * `athlete` -> `name`
    * `age place` -> `division place`
    * `age group` -> `division`
    * `gun time` -> `finish`
    * `swim-swim` -> `swim`
    * `bike-bike` or `bike-enter2` -> `bike`
    * `run-run` or `run-finish` -> `run`
    * `t1-exit1` to `t1`
    * `t2-exit2` to `t2`

Define a function to process (individual) column header names as above; first create dictionary, then define function:

In [607]:
# Do the same thing but with a dictionary to set up values to substitute.
# Define the dictionary first, and then use it in a function.
header_replacement = {
    'div place': 'division place',
    'div': 'division',
    'bib': 'no.',
    'athlete': 'name',
    'age place': 'division place',
    'age group': 'division',
    'gun time': 'finish',
    'swim-swim': 'swim',
    'bike-bike': 'bike',
    'bike-enter2': 'bike',
    'run-run': 'run',
    'run-finish': 'run',
    't1-exit1': 't1',
    't2-exit2': 't2'
}

In [608]:
def process_header_names(df):
    # get() looks up dictionary value; if not present, returns value listed second
    df.columns = [header_replacement.get(element, element) for element in list(df.columns)] 
    return(df)

In [609]:
for race in list(race_info.keys()):
    race_info[race]['df'] = process_header_names(race_info[race]['df'])

Process both tables (again, this should be done more programmatically when more tables are added):

### Add variables: decimal times

Define functions to convert times to decimal minutes:

In [610]:
def time_split_min(t):
    """Convert a time in the format hh:mm:ss into total minutes"""
    if t.count(":") == 2 and len(t) == 8:
        (h, m, s) = t.split(':')
    elif t.count(":") == 1 and len(t) == 5:
        (m, s) = t.split(':')
        h = 0
    else:
        return
    result = int(h) * 60 + int(m) + int(s)/60
    return(result)

Dictionary of replacement column names (`_hhmmss` suffix)

In [611]:
sport_col_time_names = {'finish': 'finish_hhmmss', 
                    'swim': 'swim_hhmmss', 
                    'bike': 'bike_hhmmss', 
                    'run': 'run_hhmmss', 
                    't1': 't1_hhmmss', 
                    't2': 't2_hhmmss'}

Function to rename original columns with `_hhmmss` suffix and add new columns with decimal minutes:

In [613]:
def add_decimal_minute_columns(df):
    # Add _hhmmss to original names using function above (replace original names with _hhmmss names)
    df.columns = [sport_col_time_names.get(element, element) for element in list(df.columns)] 
    # Add new columns with decimal times
    for bare, hhmmss in sport_col_time_names.items():
        if hhmmss in df.columns:
            df[bare] = [time_split_min(element) for element in df[hhmmss]]
    # Return processed dataframe
    return(df)

Apply above functions to dataframes

In [614]:
for race in list(race_info.keys()):
    race_info[race]['df'] = add_decimal_minute_columns(race_info[race]['df'])

### Get `gender` from `division`

In [615]:
def add_gender_from_division(df):
    df['gender'] = ['female' if element[0].lower() == 'f' else 'male' for element in df["division"]]

In [616]:
for race in list(race_info.keys()):
    race_info[race]['df']['gender'] = ['female' if element[0].lower() == 'f' else 'male' for element in race_info[race]['df']["division"]]

## Write to file

In [617]:
for race in list(race_info.keys()):
    race_info[race]['df'].to_csv(''.join(['../Data_output/results_', race_info[race]['dfname'], '.csv']), index = False)

## Backup code (in the embarrassing absence of version control...)

In [599]:
# # Original version; works when first line of header has all the information I need.
# def get_rr_table(url):
#     page = requests.get(url)
#     tree = html.fromstring(page.content)
    
#     for table in tree.xpath('//table[@class="results-listing__table table table-hover table-striped"]'):
#         # Get first row of header
#         # Note that race roster has two rows of the header for the individual sports, 
#         #   but they're redundant, so I ignore the second.
#         #   If the format of the table changed, this could possibly break.
#         header = [th.text_content().strip().lower() for th in table.xpath('//thead/tr[1]/th')]
#         # Get body of table; remove white spaces and \n; only keep the rows that are the right number of elements
#         data = [[td.text_content() for td in tr.xpath('td')]  
#                 for tr in table.xpath('//tr')]
#         for lst in np.arange(start = 0, stop = (len(data)), step = 1):
#             data[lst] = [element.strip() for element in data[lst]]
#         data = [row for row in data if len(row)==len(header)]
#         # Convert to pandas dataframe
#         data = pd.DataFrame(data, columns=header)
#         return(data)  # Return pandas dataframe

In [239]:
def thify(value):
    lastchar = int('{0}'.format(value)[-1])
    if (lastchar == 1):
        suffix = 'st'
    elif (lastchar == 2):
        suffix = 'nd'
    elif (lastchar == 3):
        suffix = 'rd'
    else:
        suffix = 'th'
    word = '{0}{1}'.format(value, suffix)
    return(word)

In [242]:
rockwood18sprint = pd.read_csv('../Data_output/results_rockwood18sprint.csv')
race_options = {
    'Rockwood': {
        '2017': ["rockwood17sprint"],
        '2018': ["rockwood18sprint"]
    },
    'Hampton': {
        '2017': ["hampton17sprint"],
        '2018': ["hampton18sprint"]
    }
}

selected_race_left = 'Rockwood'
selected_year_left = '2018'
df_left = eval(race_options[selected_race_left][selected_year_left][0])

selected_athlete_left = 'Lindsay Brin'
selected_sport = 'swim'
selected_dist_type_left = 'Gender'
selected_dist_value_left = 'Female'

athlete_time_left = df_left[df_left['name']==selected_athlete_left][selected_sport].values[0]
all_times_left = df_left[df_left[selected_dist_type_left.lower()]==selected_dist_value_left.lower()][selected_sport]

athlete_rank_left = (all_times_left < athlete_time_left).values.sum() + 1
athlete_percentile_left = int(round((1 - athlete_rank_left/len(all_times_left))*100))

"{0} ({1} {2}): {3} percentile ({4} out of {5})".format(
    selected_athlete_left,
    selected_race_left,
    selected_year_left, 
    thify(athlete_percentile_left),
    athlete_rank_left,
    len(all_times_left))

'Lindsay Brin (Rockwood 2018): 29th percentile (22 out of 31)'